### Pandas

대부분의 데이터는 DB에서 원본 데이터를 가져와서 전처리를 해줘야 하는데 데이터의 크기가 너무 크게 되면 전처리가 시간이 너무 길어지기 떄문에 판다스를 사용해야 합나다

#### groupby

- SQL groupby 명령어와 같음
- split > apply > combine 의 과정을 거쳐서 연산함

In [115]:
import pandas as pd
import numpy as np
import dateutil
import sqlite3

In [3]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data"
data = pd.read_csv(data_url, sep="\s+", header=None)
data.columns = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO"," B" ,"LSTAT", "MEDV"]
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [32]:
# RAD를 기준으로 묶어서 AGE를 모두 더한 값
data.groupby(["RAD"])["AGE"].sum()

RAD
1       900.5
2      1554.5
3      1873.8
4      6692.6
5      7960.7
6      1563.7
7       682.4
8      1616.4
24    11854.3
Name: AGE, dtype: float64

#### Multi index

In [33]:
# RAD와 CHAS를 기준으로 AGE의 값들은 모두 더한 멀티 인덱스(두개 이상의 인덱스) 값을 가지는 데이터프레임 출력
ex = data.groupby(["RAD", "CHAS"])["AGE"]
ex

#### Hierarchical index - ```unstack()```

In [34]:
# unstack() 함수를 사용하여 1번째 멀티 인덱스를 column으로 변환시켜 줍니다
ex.sum().unstack()

CHAS,0,1
RAD,,
1,875.7,24.8
2,1554.5,NaN
3,1760.5,113.3
4,6189.5,503.1
5,7017.5,943.2
6,1563.7,NaN
7,682.4,NaN
8,1215.9,400.5
24,11126.7,727.6


#### Hierarchical index - ```swaplevel()```

In [35]:
ex.sum().swaplevel()

CHAS  RAD
0     1        875.7
1     1         24.8
0     2       1554.5
      3       1760.5
1     3        113.3
0     4       6189.5
1     4        503.1
0     5       7017.5
1     5        943.2
0     6       1563.7
      7        682.4
      8       1215.9
1     8        400.5
0     24     11126.7
1     24       727.6
Name: AGE, dtype: float64

#### Hierarchical index - operations

In [42]:
# 0번째 수준의 인덱스를 모두 더합니다
ex.sum().sum(level=0)

RAD
1       900.5
2      1554.5
3      1873.8
4      6692.6
5      7960.7
6      1563.7
7       682.4
8      1616.4
24    11854.3
Name: AGE, dtype: float64

In [44]:
# 1번째 수준의 인덱스를 모두 더합니다
ex.sum().sum(level=1)

CHAS
0    31986.4
1     2712.5
Name: AGE, dtype: float64

In [47]:
# groupby에 의해 split된 상태를 추출 가능합니다
for name, group in ex:
    print(name)
    print(group)

(1, 0)
0      65.2
193     9.9
194    18.8
254    32.0
255    19.1
284    20.8
285    31.9
286    31.5
331    28.4
332    23.3
341    49.3
342    59.7
349    34.5
350    44.4
501    69.1
502    76.7
503    91.0
504    89.3
505    80.8
Name: AGE, dtype: float64
(1, 1)
283    24.8
Name: AGE, dtype: float64
(2, 0)
1      78.9
2      61.1
56     35.7
88     86.3
89     63.1
90     66.1
91     73.9
95     57.8
96     69.6
97     76.0
98     36.9
99     62.5
120    69.7
121    84.1
122    92.9
123    97.0
124    95.8
125    88.4
126    95.6
196    34.1
197    36.6
198    38.3
201    38.4
202    15.7
Name: AGE, dtype: float64
(3, 0)
3      45.8
4      54.2
5      58.7
39     21.8
40     15.8
41      2.9
42      6.6
43      6.5
44     40.0
45     33.8
46     33.3
47     85.5
48     95.3
49     62.0
54     47.6
64     59.5
84     48.0
85     56.1
86     45.1
87     56.8
179    58.4
180    83.3
181    62.2
182    92.2
183    95.6
184    89.8
185    68.8
186    53.6
199    15.3
200    13.9
256   

In [53]:
# 특정 key값을 가진 그룹의 정보만 추출 가능합니다
ex.get_group(name)

356    97.4
357    91.0
358    83.4
363    89.0
364    82.9
369    96.8
370    97.5
372    89.6
Name: AGE, dtype: float64

#### groupby - grouped

- 추출된 group 정보에는 세 가지 유형의 apply가 가능합니다
    1. Aggregation : 요약된 통계정보를 추출해 줍니다
    2. Transformation ; 해당 정보를 변환해 줍니다
    3. Filteration : 특정 정보를 제거하여 보여주느 필터링 기능을 합니다

#### Aggregation

- Aggregation은 key값 별로 요약된 정보를 출력합니다

In [60]:
# 기존의 sum 함수를 사용하는 것과 동일합니다
data.groupby("RAD").agg(sum) 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
RAD,,,,,,,,,,,,,
1,0.72057,798.0,101.32,1,9.2578,131.117,900.5,120.5565,5829.0,351.3,7785.46,147.40,487.3
2,1.99885,490.0,231.14,0,11.6380,159.599,1554.5,98.3282,6256.0,415.0,9273.93,240.59,644.0
3,3.69966,622.5,167.92,2,17.1921,245.975,1873.8,195.5667,9351.0,690.4,14911.88,344.89,1061.3
4,43.32938,1620.5,1182.13,8,55.4742,674.643,6692.6,487.6300,36958.0,2105.0,42099.34,1341.90,2352.6
5,79.09604,1277.5,1122.38,11,65.6516,735.213,7960.7,425.1928,38162.0,1902.9,42457.13,1225.58,2956.3
6,3.90140,337.5,213.15,0,13.3860,158.723,1563.7,104.6478,9695.0,463.2,10071.53,319.95,545.4
7,2.55679,454.0,85.59,0,7.4970,113.012,682.4,110.4296,5175.0,312.9,6603.36,135.80,460.8
8,8.91383,150.0,142.38,5,11.8200,166.819,1616.4,105.8545,7230.0,431.4,9246.66,191.06,728.6
24,1684.22640,0.0,2389.20,8,88.7590,794.924,11854.3,272.0855,87912.0,2666.4,38027.77,2455.28,2165.3


In [61]:
# lambda 함수도 사용가능합니다
data.groupby("RAD").agg(np.mean)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
RAD,,,,,,,,,,,,,
1,0.036029,39.900000,5.066000,0.050000,0.462890,6.555850,45.025000,6.027825,291.450000,17.565000,389.273000,7.370000,24.365000
2,0.083285,20.416667,9.630833,0.000000,0.484917,6.649958,64.770833,4.097008,260.666667,17.291667,386.413750,10.024583,26.833333
3,0.097359,16.381579,4.418947,0.052632,0.452424,6.473026,49.310526,5.146492,246.078947,18.168421,392.417895,9.076053,27.928947
4,0.393903,14.731818,10.746636,0.072727,0.504311,6.133118,60.841818,4.433000,335.981818,19.136364,382.721273,12.199091,21.387273
5,0.687792,11.108696,9.759826,0.095652,0.570883,6.393157,69.223478,3.697329,331.843478,16.546957,369.192435,10.657217,25.706957
6,0.150054,12.980769,8.198077,0.000000,0.514846,6.104731,60.142308,4.024915,372.884615,17.815385,387.366538,12.305769,20.976923
7,0.150399,26.705882,5.034706,0.000000,0.441000,6.647765,40.141176,6.495859,304.411765,18.405882,388.432941,7.988235,27.105882
8,0.371410,6.250000,5.932500,0.208333,0.492500,6.950792,67.350000,4.410604,301.250000,17.975000,385.277500,7.960833,30.358333
24,12.759291,0.000000,18.100000,0.060606,0.672417,6.022152,89.805303,2.061254,666.000000,20.200000,288.089167,18.600606,16.403788


In [63]:
# 한번에 여러개의 함수를 사용할 때는 list로 묶어서 사용합니다
data.groupby("RAD")["AGE"].agg([np.mean, np.sum, np.std])

,mean,sum,std
RAD,,,
1,45.025000,900.5,25.702035
2,64.770833,1554.5,23.615958
3,49.310526,1873.8,25.292791
4,60.841818,6692.6,30.837227
5,69.223478,7960.7,26.997789
6,60.142308,1563.7,24.098003
7,40.141176,682.4,26.717505
8,67.350000,1616.4,21.092859
24,89.805303,11854.3,12.625809


#### Transformation

- transformation 은 개별 데이터의 변환을 지원합니다

In [69]:
# 각 변수의 최댓값으로 변환시켜 줍니다
score = lambda x:(x.max())
data.groupby("RAD").transform(score)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,0.10959,90.0,11.93,1.0,0.573,7.923,91.0,9.2203,422.0,21.0,396.9,12.93,50.0
1,0.38735,85.0,25.65,0.0,0.581,8.069,97.0,9.1876,348.0,19.1,396.9,27.26,43.8
2,0.38735,85.0,25.65,0.0,0.581,8.069,97.0,9.1876,348.0,19.1,396.9,27.26,43.8
3,0.29916,95.0,6.96,1.0,0.488,7.831,95.6,9.2229,469.0,21.1,396.9,30.81,50.0
4,0.29916,95.0,6.96,1.0,0.488,7.831,95.6,9.2229,469.0,21.1,396.9,30.81,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.10959,90.0,11.93,1.0,0.573,7.923,91.0,9.2203,422.0,21.0,396.9,12.93,50.0
502,0.10959,90.0,11.93,1.0,0.573,7.923,91.0,9.2203,422.0,21.0,396.9,12.93,50.0
503,0.10959,90.0,11.93,1.0,0.573,7.923,91.0,9.2203,422.0,21.0,396.9,12.93,50.0
504,0.10959,90.0,11.93,1.0,0.573,7.923,91.0,9.2203,422.0,21.0,396.9,12.93,50.0


In [73]:
# 정규화를 할 때 사용하는 transform
score = lambda x : (x - x.mean() / (x.std() + 1e-5))
data.groupby("RAD").transform(score)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,-1.409375,16.799375,1.126685,-0.223597,-5.536331,-5.821455,63.448194,1.702697,289.380361,8.251302,368.978632,2.062918,20.963660
1,-1.056999,-0.564998,6.161268,0.000000,-6.600073,-3.004627,76.157329,3.026491,237.314520,9.761213,368.184759,7.633733,18.192327
2,-1.057019,-0.564998,6.161268,0.000000,-6.600073,-2.240627,58.357329,3.026491,237.314520,9.761213,364.114759,2.523733,31.292327
3,-1.270352,-0.531612,0.090359,-0.232570,-17.321253,-3.033139,43.850413,3.022584,218.137992,-5.165945,331.383120,1.293681,30.045051
4,-1.233672,-0.531612,0.090359,-0.232570,-17.321253,-2.884139,52.250413,3.022584,218.137992,-5.165945,333.653120,3.683681,32.845051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-1.353065,-1.200625,10.746685,-0.223597,-5.501331,-5.803455,67.348194,0.091297,266.380361,13.951302,364.068632,6.752918,19.363660
502,-1.370425,-1.200625,10.746685,-0.223597,-5.501331,-6.276455,74.948194,-0.099803,266.380361,13.951302,368.978632,6.162918,17.563660
503,-1.354935,-1.200625,10.746685,-0.223597,-5.501331,-5.420455,89.248194,-0.219803,266.380361,13.951302,368.978632,2.722918,20.863660
504,-1.306105,-1.200625,10.746685,-0.223597,-5.501331,-5.602455,87.548194,0.001597,266.380361,13.951302,365.528632,3.562918,18.963660


#### Filter

- 특정 조건으로 데이터를 검색할 때 사용합니다
- filter 안에는 boolean 조건이 존재해야 합니다

In [76]:
# len(x)는 grouped된 dataframe의 개수를 말합니다
data.groupby("RAD").filter(lambda x: len(x) >= 3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [78]:
data.groupby("RAD").filter(lambda x: x["AGE"].max() >= 50)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


#### Pivot table

In [79]:
!wget https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv

--2021-06-10 10:22:59--  https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv
Resolving www.shanelynn.ie (www.shanelynn.ie)... 104.236.88.249
Connecting to www.shanelynn.ie (www.shanelynn.ie)|104.236.88.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40576 (40K) [text/csv]
Saving to: 'phone_data.csv'

     0K .......... .......... .......... .........            100%  156K=0.3s

2021-06-10 10:23:01 (156 KB/s) - 'phone_data.csv' saved [40576/40576]



In [80]:
phone_data = pd.read_csv("./phone_data.csv")
phone_data.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [82]:
phone_data["date"] = phone_data["date"].apply(dateutil.parser.parse, dayfirst=True)
phone_data.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [83]:
phone_data.pivot_table(["duration"], 
                      index=[phone_data.month, phone_data.item],
                      columns=phone_data.network,
                      aggfunc="sum",
                      fill_value=0)

duration                                                                 
network        Meteor Tesco  Three Vodafone      data landline special voicemail world
month   item                                                                          
2014-11 call     1521  4045  12458     4316     0.000     2906       0       301     0
        data        0     0      0        0   998.441        0       0         0     0
        sms        10     3     25       55     0.000        0       1         0     0
2014-12 call     2010  1819   6316     1302     0.000     1424       0       690     0
        data        0     0      0        0  1032.870        0       0         0     0
        sms        12     1     13       18     0.000        0       0         0     4
2015-01 call     2207  2904   6445     3626     0.000     1603       0       285     0
        data        0     0      0        0  1067.299        0       0         0     0
        sms        10     3     33       40     0.000        0       0         0     0
2015-02 call     1188  4087   6279     1864     0.000      730       0       268     0
        data        0     0      0        0  1067.299        0       0         0     0
        sms         1     2     11       23     0.000        0       2         0     0
2015-03 call      274   973   4966     3513     0.000    11770       0       231     0
        data        0     0      0        0   998.441        0       0         0     0
        sms         0     4      5       13     0.000        0       0         0     3

#### Crosstab

- Pivot table의 특수한 형태 입니다
- User Item Rating Matrix 등을 만들 때 사용 가능합니다
- 두 칼럼에 교차 빈도, 비율, 덧셈 등을 구할 때 사용합니다

In [87]:
pd.crosstab(index= phone_data.network, columns=phone_data.network_type, values=phone_data.item,
           aggfunc="first").fillna(0)

network_type,data,landline,mobile,special,voicemail,world
network,,,,,,
Meteor,0,0,call,0,0,0
Tesco,0,0,call,0,0,0
Three,0,0,call,0,0,0
Vodafone,0,0,call,0,0,0
data,data,0,0,0,0,0
landline,0,call,0,0,0,0
special,0,0,0,sms,0,0
voicemail,0,0,0,0,call,0
world,0,0,0,0,0,sms


#### 데이터 핸들링

In [89]:
# 각 변수별 결측값 확인
phone_data["date"].isnull().sum()

0

In [91]:
# 결측값 제거
phone_data = phone_data[phone_data["date"].isnull() == False]

In [92]:
func = lambda x: "{}+".format(x)
phone_data["network"] = phone_data["network"].map(func)
phone_data.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data+,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone+,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor+,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco+,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco+,mobile


In [102]:
phone_data["network_type"].isin(["data"])

0       True
1      False
2      False
3      False
4      False
       ...  
825    False
826    False
827     True
828    False
829    False
Name: network_type, Length: 830, dtype: bool

#### Merge

- SQL에서 많이 사용하는 Merge와 같은 기능을 합니다
- 두 개의 데이터를 하나의 컬럼(key 값)을 기준으로 합칩니다.

In [106]:
df_a = pd.DataFrame({"subject_id":[1,2,3,4,5,7], "test_score":[51,15,15,61,16,14]})
df_b = pd.DataFrame({"subject_id":[4,5,6,7,8], "first_name":["Billy", "Brian", "Bran", "Bryce", "Betty"]})
# inner join
pd.merge(df_a, df_b, on="subject_id")

,subject_id,test_score,first_name
0,4,61,Billy
1,5,16,Brian
2,7,14,Bryce


In [108]:
# 서로 같은 컬럼이지만 컬럼명이 다른 경우 
pd.merge(df_a, df_b, left_on="subject_id", right_on="subject_id")

,subject_id,test_score,first_name
0,4,61,Billy
1,5,16,Brian
2,7,14,Bryce


In [110]:
pd.merge(df_a, df_b, on="subject_id", how="right")

,subject_id,test_score,first_name
0,4,61.0,Billy
1,5,16.0,Brian
2,6,NaN,Bran
3,7,14.0,Bryce
4,8,NaN,Betty


In [112]:
pd.merge(df_a, df_b, on="subject_id", how="inner")

,subject_id,test_score,first_name
0,4,61,Billy
1,5,16,Brian
2,7,14,Bryce


In [114]:
pd.merge(df_a, df_b, on="subject_id", how="outer")

,subject_id,test_score,first_name
0,1,51.0,NaN
1,2,15.0,NaN
2,3,15.0,NaN
3,4,61.0,Billy
4,5,16.0,Brian
5,7,14.0,Bryce
6,6,NaN,Bran
7,8,NaN,Betty


#### Concat

#### DB Persistance

jupyter 에서 셀을 실행을 하면 정보들이 메모리 공간에 할당, 존재하게 됩니다. 힘들게 전처리한 데이터를 다른 시간에 데이터를 파일, 고정시켜주기 위한 방법으로 db 형태로 만드는 방법도 있습니다

In [128]:
conn = sqlite3.connect("./flights.db")
cur = conn.cursor()
cur.execute("select * from airports limit 5;")
results = cur.fetchall()
results

OperationalError: no such table: airports

In [129]:
df_airline = pd.read_sql_query("select * from airlines;", conn)

DatabaseError: Execution failed on sql 'select * from airlines;': no such table: airlines

#### XLS persistence

- dataframe의 엑셀 추출 코드
- xls 엔진으로 openpyxls 또는 XlsxWrite 사용

In [130]:
writer = pd.ExcelWriter("./example.xlsx", engine="xlsxwriter")
phone_data.to_excel(writer, sheet_name="Sheet1")

#### Pickle persistence

- 가장 일반적인 python persistence
- to_pickel, read_pickle 함수 이용